In [3]:
# Importing all library needed
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import json
import time

# starting scraping process
url =  "https://en.wikipedia.org/wiki/Big_Tech"
response = requests.get(url)
soup = bs(response.text, "html.parser")

#finding all table on the website to scrape with the class listed below
table = soup.find_all("table", {"class": "wikitable sortable mw-collapsible floatright"})

# Read the html table and put the dataframe in a list
df = pd.read_html(str(table))


# Getting the 1st DF named 'Big Five tech companies'
tb1 =df[0]
# Getting the 2nd DF named 'Smaller U.S. Big Tech companies'
tb2 =df[1]

# Renaming the column since they're spell different so we can merge them 
df1_renamed = tb1.rename(columns={"Revenue (USD)[29]": "Revenue (USD)"})
df2_renamed = tb2.rename(columns={"Revenue (USD)[90]": "Revenue (USD)"})


#---- Creating 1st DataFrame ----
#Combine 2 tech company DF into one 
DF1 = pd.concat([df1_renamed,df2_renamed])

# Resetting index so it can be merge with DF2
DF1.reset_index(drop=True, inplace=True)
DF1

,Company,Revenue (USD)[10],Profit (USD),Subsidiaries,Revenue (USD)[93]
0,Alphabet,$283 billion,$60 billion,GoogleGVWaymoX,NaN
1,Amazon,$514 billion,$-3 billion,AudibleiRobotTwitchWhole Foods,NaN
2,Apple,$394 billion,$99 billion,Beats,NaN
3,Meta,$116 billion,$23 billion,FacebookInstagramReality LabsWhatsApp,NaN
4,Microsoft,$212 billion,$73 billion,GitHubLinkedInSkype,NaN
5,IBM,NaN,2 billion,Red Hat,60 billion
6,Tesla,NaN,12 billion,—,81 billion
7,Oracle,NaN,8 billion,Cerner,50 billion
8,Netflix,NaN,5 billion,—,31 billion
9,Nvidia,NaN,4 billion,—,27 billion


In [4]:
#----Scraping S&P500 wikipedia table for stock ticker symbol----

# Instantiating a variable for company name and ticker symbol
CompNameList = DF1["Company"].tolist()
symbol_list = []
security_list =[]

stockurl =  "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500response = requests.get(stockurl)
sp500table= bs(sp500response.text, "html.parser")

# find all row in the table and loop through it
rows = sp500table.find_all("tr")
for row in rows:
    
# find all tabledata in the table in each row
    td_tags = row.find_all("td")
    
# Checking to see if the list length is greater than 0
# If yes we get the text and remove the "\n" from the string
    if len(td_tags) > 0:
        symbol = td_tags[0].get_text().strip("\n")
        security = td_tags[1].get_text().strip("\n")
        
# Taking the first word in the string
        new_security = security.split(" ")[0]
        
        for name in CompNameList:
# Looking for the first occurance of the stock in the table
            if name in security and  name not in security_list:
                security_list.append(new_security)
                symbol_list.append(symbol)

print(symbol_list)
print(security_list)

# Creating DF for symbol and security
ss_df = pd.DataFrame({"Ticker Symbol":symbol_list, "Security Name":security_list})
ss_df

['ADBE', 'GOOGL', 'AMZN', 'AAPL', 'IBM', 'META', 'MSFT', 'NFLX', 'NVDA', 'ORCL', 'CRM', 'TSLA']
['Adobe', 'Alphabet', 'Amazon', 'Apple', 'IBM', 'Meta', 'Microsoft', 'Netflix', 'Nvidia', 'Oracle', 'Salesforce', 'Tesla,']


,Ticker Symbol,Security Name
0,ADBE,Adobe
1,GOOGL,Alphabet
2,AMZN,Amazon
3,AAPL,Apple
4,IBM,IBM
5,META,Meta
6,MSFT,Microsoft
7,NFLX,Netflix
8,NVDA,Nvidia
9,ORCL,Oracle


In [2]:
# Creating function that return the stock information in dataframe of the stock's ticker symbol
def stock_Info(Ticker):
    
# Calling api for information 
    key="487EQ5T70C156H7X"
    infourl = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={Ticker}&interval=5min&apikey={key}"
    r = requests.get(infourl)
    data = r.json()
    cleanData = json.dumps(data, sort_keys=True, indent=4)

    # instantiating a variable for json data (DF list)
    opening_Price = []
    high_Price = []
    low_Price = []
    closing_Price = []
    volume_Traded = []

    Ticker_list =[]
    op_list = []
    hp_list = []
    lp_list = []
    cp_list = []
    vt_list = []
    
# Looping through the json to get the value 
    for timestamp, values in data['Time Series (5min)'].items():
            op_list.append(float(values["1. open"]))
            hp_list.append(float(values["2. high"]))
            lp_list.append(float(values["3. low"]))
            cp_list.append(float(values["4. close"]))
            vt_list.append(int(values["5. volume"]))



# Appending the value in the list to the DF list so we cna create DF
    # finding the opening price and append it to the open_price(DF) list
    opening_Price.append(op_list[0])

    # finding the highest price and append it to the high_price(DF) list

    high_Price.append(max(hp_list))

    # finding the lowest price and append it to the low_price(DF) list
    low_Price.append(min(lp_list))

    # finding the price at the end of the time stamp and append it to close_price(DF) List
    closing_Price.append(cp_list[-1])

    # finding the volume at the end of the time stamp and append it to volume_Traded(DF) List
    volume_Traded.append(sum(vt_list))

    # adding the symbol for stock infomation to symbol_list(DF) List
    Ticker_list.append(data["Meta Data"]["2. Symbol"])

# Creating DataFrame with stock info at the current ticker symbol
    info_df = pd.DataFrame({"Ticker Symbol":Ticker_list, "Opening Price":opening_Price, "High Price":high_Price, "Lowest Price": low_Price,"volume Trade": volume_Traded })
    return info_df

In [5]:
# adding DataFrame to a list
df_list = []

# Looping through the list of ticker symbol to get stock info
for ticker in symbol_list:
    df_list.append(stock_Info(ticker))
    print(df_list)
    
# Waiting for 12 second to make another api call (since I can only make 5 call in 60 second)
    time.sleep(60/5)
    
#combining the list of DF into one dataframe
combined_df = pd.concat(df_list)
combined_df

[  Ticker Symbol  Opening Price  High Price  Lowest Price  volume Trade
0          ADBE         510.01      518.09       499.595       2414640]
[  Ticker Symbol  Opening Price  High Price  Lowest Price  volume Trade
0          ADBE         510.01      518.09       499.595       2414640,   Ticker Symbol  Opening Price  High Price  Lowest Price  volume Trade
0         GOOGL         130.88     138.007        130.36      26383300]
[  Ticker Symbol  Opening Price  High Price  Lowest Price  volume Trade
0          ADBE         510.01      518.09       499.595       2414640,   Ticker Symbol  Opening Price  High Price  Lowest Price  volume Trade
0         GOOGL         130.88     138.007        130.36      26383300,   Ticker Symbol  Opening Price  High Price  Lowest Price  volume Trade
0          AMZN         127.01      131.88       119.166      46057293]
[  Ticker Symbol  Opening Price  High Price  Lowest Price  volume Trade
0          ADBE         510.01      518.09       499.595       2414

,Ticker Symbol,Opening Price,High Price,Lowest Price,volume Trade
0,ADBE,510.01,518.090,499.595,2414640
0,GOOGL,130.88,138.007,130.360,26383300
0,AMZN,127.01,131.880,119.166,46057293
0,AAPL,171.33,181.657,164.292,43635268
0,IBM,140.58,141.730,139.610,9616533
0,META,300.25,307.240,299.360,18163176
0,MSFT,316.00,318.330,294.558,22056131
0,NFLX,377.65,378.797,375.350,3183071
0,NVDA,435.09,438.200,418.668,23713935
0,ORCL,105.90,106.465,105.380,10385308


In [6]:
#---- Creating 2nd DataFrame ----

# Creating DF2 with 2 dataframe by matching the ticker symbol
DF2 = pd.merge(combined_df, ss_df, on="Ticker Symbol")
DF2

,Ticker Symbol,Opening Price,High Price,Lowest Price,volume Trade,Security Name
0,ADBE,510.01,518.090,499.595,2414640,Adobe
1,GOOGL,130.88,138.007,130.360,26383300,Alphabet
2,AMZN,127.01,131.880,119.166,46057293,Amazon
3,AAPL,171.33,181.657,164.292,43635268,Apple
4,IBM,140.58,141.730,139.610,9616533,IBM
5,META,300.25,307.240,299.360,18163176,Meta
6,MSFT,316.00,318.330,294.558,22056131,Microsoft
7,NFLX,377.65,378.797,375.350,3183071,Netflix
8,NVDA,435.09,438.200,418.668,23713935,Nvidia
9,ORCL,105.90,106.465,105.380,10385308,Oracle
